# Import libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.signal as ss
import matplotlib
import matplotlib.pyplot as plt
import os

from mpdaf.obj import Cube
from mpdaf.sdetect import Source

from deblend import main_deblending

# Import data

In [ ]:
DIR_SRC='./data/'
ID = 6506
src = Source.from_file(DIR_SRC+'udf_mosaic_%s.fits'%str(ID).zfill(5))

## Deblend 

In [ ]:
# create main object initializing with the source
debl=main_deblending.Deblending(src)

#create the intensity maps using segmap in source (any 2d array can be used)
debl.createIntensityMap(segmap=src.images['HST_SEGMAP'].data)

# do the deblending. The following parameters are the default ones
# transfert_hst : compute transfert function from HST to MUSE so there is no need to apply HST psf on MUSE data
# antialias : as the convolution on HST is done before the subsampling there is no need for 
# antialias filter when subsampling HST, thus no need to apply it also on MUSE data
# regul: use regularization or not

debl.findSources(transfert_hst=True,antialias=False,regul=False)


## Explore results

### Segmap

In [ ]:
plt.imshow(debl.labelHR)

### Spectra

The list of spectra (calibrated in flux) is stored in debl.sources and their variances are stored in debl.varSources. It can also be obtained as a dict of mpdaf spectras with the HST IDs as keys using debl.getsp() (with the key 'bg' for the background spectrum).

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(src.spectra['MUSE_WHITE_SKYSUB'].data)

In [ ]:
# Compare spectra of object 3 with and without regul
plt.plot(debl_regul.sources[3])
plt.plot(debl_noregul.sources[3],alpha=0.5)

### Intensity maps

The map of objects segmentation map with intern IDs is stored in debl.labelHR (nd array). The same with HST IDs is stored in debl.segmap

In [ ]:
plt.imshow(debl_regul.listIntensityMapLRConvol[0][1][2].reshape(debl_regul.shapeLR))

### Residuals

Residuals (data-reconstructed) are stored in debl.residuals, rebuilt cube is stored in cubeRebuilt

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(121)
_min, _max = np.amin(debl_regul.cubeLR.sum(axis=0)), np.amax(debl_regul.cubeLR.sum(axis=0))
plt.imshow(debl_regul.cubeLR.sum(axis=0),vmin=_min,vmax=_max)
plt.colorbar(fraction=0.046)
plt.subplot(122)
plt.imshow(debl_regul.cubeRebuilt.sum(axis=0),vmin=_min,vmax=_max)
plt.colorbar(fraction=0.046)

## Analyze results

In [ ]:
from deblend import eval_utils

In [ ]:
print "Intercorrelation between the two main objects"
print "without regularization",eval_utils.calcInterCorr2(debl_noregul.sources[3],debl_noregul.sources[2])
print "with regularization",eval_utils.calcInterCorr2(debl_regul.sources[3],debl_regul.sources[2])
